# Themes

> Load and format SRF, GCM, and cross-cutting priority data for LLM consumption


In [ ]:
#| default_exp themes

In [ ]:
#| export
from fastcore.all import *
from pathlib import Path
import json

In [ ]:
#| export
def _load(fname:str, # Filename to load
          path:str='files/themes', # Directory containing theme files
          md:bool=False # Return raw text instead of JSON?
         ) -> dict|str:
    "Load theme file (JSON or markdown)"
    p = Path(path)/fname
    return p.read_text() if md else json.loads(p.read_text())

In [ ]:
#| export
def load_enablers(path:str='files/themes' # Directory containing theme files
                 ) -> list:
    "Load SRF enablers"
    return _load('srf_enablers.json', path)

In [ ]:
#| export
def load_ccp(path:str='files/themes' # Directory containing theme files
            ) -> list:
    "Load cross-cutting priorities"
    return _load('crosscutting_priorities.json', path)

In [ ]:
#| export
def load_srf_outs(path:str='files/themes' # Directory containing theme files
                 ) -> list:
    "Load SRF objectives with outputs"
    return _load('srf_objectives.json', path)

In [ ]:
#| export
def load_gcms(path:str='files/themes' # Directory containing theme files
             ) -> str:
    "Load GCM objectives markdown"
    return _load('gcms_long.md', path, md=True)

In [ ]:
#| export
def load_gcm_lut(path:str='files/themes' # Directory containing theme files
                ) -> dict:
    "Load GCM to SRF output lookup table"
    return _load('gcm_to_srf_outputs.json', path)

In [ ]:
enablers = load_enablers()
ccp = load_ccp()
srf_outs = load_srf_outs()
gcms = load_gcms()
lut = load_gcm_lut()

print(gcms[:200])
len(enablers), len(ccp), len(srf_outs), len(lut)


# Objective 1
## Title: Collect and utilize accurate and disaggregated data as a basis for evidence-based policies
## Associated actions
- a. Elaborate and implement a comprehensive strategy for impro


(7, 4, 3, 23)

## SRF Formatting

Format SRF enablers, cross-cutting priorities, and outputs for LLM consumption with full hierarchical context. These functions produce markdown with headers and descriptions suitable for including in prompts.

In [ ]:
#| export
def fmt_enablers_ccp(items:list # List of enabler/CCP dicts with id, title, description
                    ) -> str:
    "Format enablers or cross-cutting priorities for LLM"
    return '\n\n'.join([f'## Cross-cutting {o["id"]}: {o["title"]}\n### Description\n{o["description"]}' for o in items])

In [ ]:
enablers = load_enablers()
print(fmt_enablers_ccp(enablers[:2])[:400])

## Cross-cutting 1: Workforce
### Description
IOM’s diverse and capable people are our most valued asset. Through investing in better workforce planning and people management, we will facilitate their professional development and improve their daily workplace experience. IOM will have flexible systems and procedures in place to ensure it can adapt to the future of work while ensuring the safety, s


In [ ]:
#| export
def get_srf_out(objectives:list, # SRF objectives structure
                output_id:str # Output ID to find
               ) -> dict|None:
    "Retrieve single SRF output with hierarchy"
    for obj in objectives:
        for lt_out in obj.get('long_term_outcomes', []):
            for st_out in lt_out.get('short_term_outcomes', []):
                if o := next((o for o in st_out.get('outputs', []) if o['id'] == output_id), None):
                    return dict(obj_id=obj['id'], obj=obj['title'], lt_out_id=lt_out['id'], lt_out=lt_out['title'],
                                st_out_id=st_out['id'], st_out=st_out['title'], output=o)

In [ ]:
srf_outs = load_srf_outs()
out = get_srf_out(srf_outs, '1a11')
out['output']['title']

'Crisis-affected populations in-need receive dignified shelter and settlement support.'

In [ ]:
#| export
def fmt_srf_out(output_ctx:dict # Dict with obj, lt_out, st_out, and output fields
               ) -> str:
    "Format SRF output with hierarchical context for LLM"
    o = output_ctx
    return '\n'.join([
        f'## SRF Output {o["output"]["id"]}: {o["output"]["title"]}',
        '### Hierarchical Context',
        f'**Objective {o["obj_id"]}**: {o["obj"]}',
        f'**Long-term Outcome {o["lt_out_id"]}:** {o["lt_out"]}',
        f'**Short-term Outcome {o["st_out_id"]}:** {o["st_out"]}'
    ])

In [ ]:
print(fmt_srf_out(out))

## SRF Output 1a11: Crisis-affected populations in-need receive dignified shelter and settlement support.
### Hierarchical Context
**Objective 1**: Saving lives and protecting people on the move.
**Long-term Outcome 1a:** Human suffering is alleviated while the dignity and rights of people affected by crises are upheld.
**Short-term Outcome 1a1:** Crisis-affected populations have their basic needs met and have minimum living conditions with reduced barriers to access for marginalized and vulnerable individuals.


In [ ]:
#| export
def fmt_srf_outs(objectives:list, # SRF objectives structure
                 output_ids:list # List of output IDs to format
                ) -> str:
    "Format multiple SRF outputs"
    return '\n\n'.join(fmt_srf_out(get_srf_out(objectives, i)) for i in output_ids)


In [ ]:
print(fmt_srf_outs(srf_outs, ['1a11', '1a31']))

## SRF Output 1a11: Crisis-affected populations in-need receive dignified shelter and settlement support.
### Hierarchical Context
**Objective 1**: Saving lives and protecting people on the move.
**Long-term Outcome 1a:** Human suffering is alleviated while the dignity and rights of people affected by crises are upheld.
**Short-term Outcome 1a1:** Crisis-affected populations have their basic needs met and have minimum living conditions with reduced barriers to access for marginalized and vulnerable individuals.

## SRF Output 1a31: Guidelines on data and information collection, sharing and management are in place that adhere to data protection standards, principles of confidentiality and a defined purpose, to protect the individuals and groups providing information from harm.
### Hierarchical Context
**Objective 1**: Saving lives and protecting people on the move.
**Long-term Outcome 1a:** Human suffering is alleviated while the dignity and rights of people affected by crises are uphel

## GCM Functions

Link Global Compact for Migration (GCM) objectives to SRF outputs. The lookup table maps GCM objective IDs to their corresponding SRF output IDs.

In [ ]:
#| export
def get_srf_outs(lut:dict, # GCM to SRF lookup dict
                 gcm_ids:list # List of GCM IDs to filter by
                ) -> L:
    "Get SRF output IDs filtered by GCM IDs"
    return L(v for k,v in lut.items() if k in gcm_ids).concat()

In [ ]:
gcm_lut = load_gcm_lut()
out_ids = get_srf_outs(gcm_lut, ['1', '4'])
out_ids[:5]

(#5) ['1a31','1a32','1b11','3b31','3c13']

## Convenience Functions

Load all themes at once for quick setup.

In [ ]:
#| export
def load_all(path:str='files/themes' # Directory containing theme files
            ) -> dict:
    "Load all theme data"
    return dict(enablers=load_enablers(path), ccp=load_ccp(path), gcms=load_gcms(path), srf_outs=load_srf_outs(path), gcm_lut=load_gcm_lut(path))

In [ ]:
all_themes = load_all()
all_themes.keys()

dict_keys(['enablers', 'ccp', 'gcms', 'srf_outs', 'gcm_lut'])

In [ ]:
ccp = load_ccp()
test_eq(len(enablers), 7)
test_eq(enablers[0]['title'], 'Workforce')
test_eq(len(ccp), 4)
test_eq(len(srf_outs), 3)